#  Science Image Calibration notebook

This notebook calibrates science images by removing bias, dark and dividing out flats.

Made by: Harlan Shaw <harlan.shaw@ucalgary.ca>

## Required Python packages:
[Astropy](https://www.astropy.org/)

[CCDProc](https://ccdproc.readthedocs.io/en/latest/index.html)

[Astro-SCRAPPY](https://github.com/astropy/astroscrappy)

It's recommended you download and install [Anaconda](https://www.anaconda.com/products/individual#Downloads) as this contains a Python environment and Astropy.

You will need to install CCDProc using the Conda install command: `conda install -c conda-forge ccdproc`

This will also install Astropy and Astro-SCRAPPY if not already installed.

## Citations:
This project uses:

[Astropy](https://www.astropy.org/acknowledging.html)

[CCDProc](https://ccdproc.readthedocs.io/en/latest/citation.html)

[Astro-SCRAPPY](https://github.com/astropy/astroscrappy)


## Required Variables

The next cell has variables that must be set for the notebook to function

In [ ]:
from pathlib import Path
from astropy.wcs import FITSFixedWarning
import warnings

warnings.filterwarnings("ignore", category=FITSFixedWarning)
# Replace each point in Path() with the bias, dark, flat, mask files and data directory as below
PATH_TO_BIAS = Path("reduced/combined_bias_mean.fit")
PATH_TO_DARK = Path("reduced/combined_dark_25C_mean.fit")
PATH_TO_FLAT = Path("reduced/combined_flat.fit")
PATH_TO_MASK = Path("reduced/combined_mask.fit")
PATH_TO_DATA = Path("I:/Work/BN Data/Skynet/1x1/17C/60s")

In [ ]:
from pathlib import Path
from astropy.nddata import CCDData
from astropy.units import adu, second, dimensionless_unscaled
from ccdproc import ImageFileCollection, subtract_bias, subtract_dark, flat_correct, cosmicray_lacosmic

data_files = ImageFileCollection(PATH_TO_DATA, glob_include="*.fit*")
# Read in all fits files
combined_bias = CCDData.read(PATH_TO_BIAS, unit=adu)
combined_dark = CCDData.read(PATH_TO_DARK, unit=adu)
combined_flat = CCDData.read(PATH_TO_FLAT, unit=adu)
combined_mask = CCDData.read(PATH_TO_MASK, unit=dimensionless_unscaled)
# Take mask matrix out of CCD
mask = combined_mask.data
# Set up output directory
output_directory = PATH_TO_DATA.joinpath("corrected_b_d_f")
output_directory.mkdir(parents=True, exist_ok=True)

for file_name in data_files.files:
    file_name_stem = Path(file_name).stem
    full_path = PATH_TO_DATA.joinpath(file_name)
    # Since we're doing 3 modifications, output filename will be _b_d_f
    # for bias, dark and flat respectively
    new_filename = full_path.with_stem(file_name_stem + "_b_d_f")
    output_file = output_directory.joinpath(new_filename.name)
    # Read in datafile and apply changes
    data_ccd = CCDData.read(full_path, unit=adu)
    data_ccd = subtract_bias(data_ccd, combined_bias, add_keyword={"HISTORY": f"Subtracted master bias {PATH_TO_BIAS.name}"})
    data_ccd = subtract_dark(ccd=data_ccd, master=combined_dark, exposure_time="EXPTIME", exposure_unit=second, scale=True, add_keyword={"HISTORY": f"Subtracted master dark {PATH_TO_DARK.name}"})
    data_ccd = flat_correct(ccd=data_ccd, flat=combined_flat, add_keyword={"HISTORY": f"Divided master flat {PATH_TO_FLAT.name}"})
    #data_ccd = cosmicray_lacosmic(ccd=data_ccd, sigclip=8, gain=1.42, readnoise=14.1)
    #new_mask = data_ccd.mask | mask
    # Mask currently obliterates all "bad" pixel points by setting to 0
    #data_ccd.multiply(~new_mask)
    #data_ccd.header["HISTORY"] = f"Removed cosmic rays with LACosmic"
    data_ccd.write(output_file, output_verify="ignore")
